## Optimizers

**Definition:** An optimizer is an algorithm that adjusts the parameters of a model to minimize a given loss function during training. It aims to solve the following general optimization problem:

$$\min_{\theta\in\mathcal{D}}\mathcal{L}(\theta)$$

Where:

 - $\theta \in \mathbb{R}^d$ is the parameter vector
 - $\mathcal{L}: \mathcal{D} \subseteq \mathbb{R}^d \to \mathbb{R}$ is the objective (or loss) function,
 - $\mathcal{D}$ is the domain (possibly with constraints),
 - The goal is to find $\theta^* \in \mathcal{D}$ such that:

 $$\mathcal{L}(\theta^*)=\inf_{\theta\in\mathcal{D}}\mathcal{L}(\theta)$$


An optimizer defines an **update rule** that generates a sequence of parameter vectors:

$$
\theta^{(0)}, \theta^{(1)}, \theta^{(2)}, \dots
$$

such that ideally:

$$
\lim_{k \to \infty} \mathcal{L}(\theta^{(k)}) \to \inf_{\theta \in \mathcal{D}} \mathcal{L}(\theta)
$$

Each update follows a general form:

$$
\theta^{(k+1)} = \mathcal{A}(\theta^{(k)}, \mathcal{L}, \nabla \mathcal{L}, k, \ldots)
$$

### Optimizer Bias





#### Gradient Estimation Bias

If $\xi^{(k)}$ introduces a stochastic gradient estimate:

$$
g^{(k)} = \nabla \mathcal{L}_{\xi^{(k)}}(\theta^{(k)})
$$

Then bias is defined as:

$$
\text{Bias}[g^{(k)}] = \mathbb{E}_{\xi^{(k)}}[g^{(k)}] - \nabla \mathcal{L}(\theta^{(k)})
$$




#### Trajectory Bias

Even with unbiased gradients, the update rule may induce a biased trajectory:

$$
\theta^{(k+1)} = \theta^{(k)} + \Delta \theta^{(k)}
$$

with:

$$
\Delta \theta^{(k)} = \mathcal{F}(\{\nabla \mathcal{L}(\theta^{(j)})\}_{j \le k})
$$

If the limit $\theta^* = \lim_{k \to \infty} \theta^{(k)}$ satisfies:

$$
\theta^* \ne \arg\min_\theta \mathcal{L}(\theta)
$$

then the optimizer is **biased toward a non-optimal solution**, often due to its internal dynamics.




#### Solution Bias

Let $\Theta^* = \{\theta : \mathcal{L}(\theta) = \mathcal{L}_{\min} \}$ denote the set of global minima.

If the optimizer consistently selects a specific $\theta^* \in \Theta^*$, then it induces a **selection bias**:

$$
\theta^* = \arg\min_{\theta \in \Theta^*} R(\theta)
$$

where $R(\theta)$ is some implicit regularization (e.g., norm, sharpness), even if not explicitly part of $\mathcal{L}$.



### Gradient Descent (GD)

**Update Rule:**

$$
\theta_{t+1} = \theta_t - \eta \nabla_\theta \mathcal{L}(\theta_t)
$$

- $\eta > 0$ is the learning rate.
- Uses the full dataset to compute the exact gradient.

**Implication:** Converges smoothly to a local minimum if $\mathcal{L}$ is convex and $\eta$ is well-chosen. Computationally expensive for large datasets.




### Stochastic Gradient Descent (SGD)

**Update Rule:**

$$
\theta_{t+1} = \theta_t - \eta \nabla_\theta \mathcal{L}(\theta_t; x_i)
$$

- Uses a single sample (or mini-batch) at each step.

**Implication:** Introduces noise but allows faster iterations. Can escape shallow local minima. Requires learning rate decay or scheduling.




### Momentum

**Update Rule:**

$$
\begin{aligned}
v_{t+1} &= \mu v_t - \eta \nabla_\theta \mathcal{L}(\theta_t) \\
\theta_{t+1} &= \theta_t + v_{t+1}
\end{aligned}
$$

- $\mu \in [0,1)$ is the momentum coefficient.

**Implication:** Helps accelerate learning by smoothing out gradient updates; especially useful in ravines.




### Nesterov Accelerated Gradient (NAG)

**Update Rule:**

$$
\begin{aligned}
v_{t+1} &= \mu v_t - \eta \nabla_\theta \mathcal{L}(\theta_t + \mu v_t) \\
\theta_{t+1} &= \theta_t + v_{t+1}
\end{aligned}
$$

**Implication:** Looks ahead by evaluating the gradient at the approximate future position. Generally converges faster than standard momentum.




### Adagrad

**Update Rule:**

$$
\begin{aligned}
G_t &= G_{t-1} + \nabla_\theta \mathcal{L}(\theta_t)^2 \\
\theta_{t+1} &= \theta_t - \frac{\eta}{\sqrt{G_t + \epsilon}} \nabla_\theta \mathcal{L}(\theta_t)
\end{aligned}
$$

- Accumulates past squared gradients to adapt learning rates.

**Implication:** Works well for sparse data but suffers from aggressive learning rate decay.




### RMSprop

**Update Rule:**

$$
\begin{aligned}
E[g^2]_t &= \gamma E[g^2]_{t-1} + (1 - \gamma) (\nabla_\theta \mathcal{L}(\theta_t))^2 \\
\theta_{t+1} &= \theta_t - \frac{\eta}{\sqrt{E[g^2]_t + \epsilon}} \nabla_\theta \mathcal{L}(\theta_t)
\end{aligned}
$$

- $\gamma$ is typically set to 0.9.

**Implication:** Mitigates Adagrad's aggressive decay by using exponential moving average of squared gradients.




### Adam (Adaptive Moment Estimation)

**Update Rule:**

$$
\begin{aligned}
m_t &= \beta_1 m_{t-1} + (1 - \beta_1) \nabla_\theta \mathcal{L}(\theta_t) \\
v_t &= \beta_2 v_{t-1} + (1 - \beta_2) (\nabla_\theta \mathcal{L}(\theta_t))^2 \\
\hat{m}_t &= \frac{m_t}{1 - \beta_1^t} \quad , \quad \hat{v}_t = \frac{v_t}{1 - \beta_2^t} \\
\theta_{t+1} &= \theta_t - \eta \frac{\hat{m}_t}{\sqrt{\hat{v}_t} + \epsilon}
\end{aligned}
$$

- Typical values: $\beta_1 = 0.9$, $\beta_2 = 0.999$, $\epsilon = 10^{-8}$.

**Implication:** Combines momentum and adaptive learning rates. Excellent general-purpose optimizer.




### AdaDelta

**Update Rule:**

$$
\begin{aligned}
E[g^2]_t &= \rho E[g^2]_{t-1} + (1 - \rho) g_t^2 \\
\Delta\theta_t &= - \frac{\sqrt{E[\Delta\theta^2]_{t-1} + \epsilon}}{\sqrt{E[g^2]_t + \epsilon}} g_t \\
E[\Delta\theta^2]_t &= \rho E[\Delta\theta^2]_{t-1} + (1 - \rho)(\Delta\theta_t)^2 \\
\theta_{t+1} &= \theta_t + \Delta\theta_t
\end{aligned}
$$

- $\rho \in [0, 1)$ is typically around 0.9.

**Implication:** No need to set a learning rate manually. Effective at stabilizing updates.




### AdamW

**Update Rule:**

Similar to Adam, but decouples weight decay:

$$
\theta_{t+1} = \theta_t - \eta \left( \frac{\hat{m}_t}{\sqrt{\hat{v}_t} + \epsilon} + \lambda \theta_t \right)
$$

- $\lambda$ is the weight decay coefficient.

**Implication:** Regularizes weights more explicitly than standard Adam. Now preferred over Adam for training deep networks.


### Neuron

**Definition:** The neuron is a function $f:\mathbb{R}^n\rightarrow\mathbb{R}$ defined as 

$$f(\mathbf{x})=\phi(\mathbf{w}\cdot\bar{\mathbf{x}})$$

where

 - $\mathbf{x}\in\mathbb{R}^n$ is the input vector
 - $\bar{\mathbf{x}}=\begin{bmatrix}x\\1\end{bmatrix}\in\mathbb{R}^{n+1}$ is the input vector with a 1 for the bias term
 - $\mathbf{w}\in\mathbb{R}^{n+1}$ is the weight vector with a bias term
 - $\phi:\mathbb{R}\rightarrow\mathbb{R}$ is the activation function

**Non-linear Activation:** The Universal Approximation Theorem states, a feedforward neural network with a single hidden layer containing a finite number of neurons and a non-linear activation function can approximate any continuous function on compact subsets of $\mathbb{R}^n$.

### Single Layer Neural Network

**Definition:** A single layer neural network is a vector-valued function $f:\mathbb{R}^n\rightarrow\mathbb{R}^m$ defined by

$$f(\mathbf{x})=\phi(\mathbf{W}\bar{\mathbf{x}})$$

where

 - $\mathbf{x}\in\mathbb{R}^n$ is the input vector          w
 - $\bar{\mathbf{x}}=\begin{bmatrix}x\\1\end{bmatrix}\in\mathbb{R}^{n+1}$ is the input vector augmented with a 1 for the bias term,
 - $\mathbf{W}\in\mathbb{R}^{m\times(n+1)}$ is the weight matrix, where each row corresponds to the weights (including bias) for one neuron,
 - $\phi:\mathbb{R}\rightarrow\mathbb{R}$ is a scalar activation function, extended to vectors in $\mathbb{R}^m$ by elementwise application of the dot product.

### Multi Layer Neural Network


**Definition:** A multi layer neural network is a vector-valued function $f:\mathbb{R}^n\rightarrow\mathbb{R}^m$ defined as a composition of affine transformations and activation functions over $L$ layers:

$$f(\mathbf{x})=\phi^{(L)}(\mathbf{W}^{(L)}\cdot\phi^{(L-1)}(\mathbf{W}^{(L-1)}\ldots\phi^{(1)}(\mathbf{W}^{(1)}\cdot\bar{\mathbf{x}})\ldots))$$

where

 - $\mathbf{x}\in\mathbb{R}^n$ is the input vector.
 - $\bar{\mathbf{x}}=\begin{bmatrix}x\\1\end{bmatrix}\in\mathbb{R}^{n+1}$ is the input augmented with a bias term.
 - $L\in\mathbb{N}$ is the total number of layers.
 - $\phi^{(\ell)}:\mathbb{R}^{d_{\ell}}\rightarrow\mathbb{R}^{d_{\ell}}$ is an activation function applied component wise.
 - $\mathbf{W}^{(\ell)}\in\mathbb{R}^{d_{\ell}\times(d_{\ell-1}+1)}$ is the weight matrix of the $\ell$-th layer, where $d_0=n$ and $d_L=m$.


**Layer-wise Representation:** 

$$h^{(0)}=\mathbf{x}$$

$$\begin{align*}
\mathbf{z}^{(\ell)}&=\mathbf{W}^{(\ell)}\cdot\bar{\mathbf{h}}^{(\ell-1)}&\text{pre-activation}\\
\mathbf{h}^{(\ell)}&=\phi^{(\ell)}(\mathbf{z}^{(\ell)})&\text{post-activation}
\end{align*}$$


### Backpropagation

**Definition:** A highly efficient algorithm for indirectly computing the gradient of the loss function with respect to all the weights in a neural network by applying the chain rule in a structured, recursive way.

**Algorithm:**

*forward pass:* Compute $\mathbf{z}^{(\ell)},\mathbf{h}^{(l)}$ for $\ell=1,\ldots,L$.

*backward pass:*

 1. Initialize $\delta^{(L)}=\nabla_{\mathbf{h}^{(L)}}\mathcal{L}\circ\phi'^{(L)}$
 2. For $\ell=L-1,\ldots,1$
    - $\boldsymbol{\delta}^{[\ell]} = \left( (\mathbf{W}^{[\ell+1]})^\top \boldsymbol{\delta}^{[\ell+1]} \right) \circ \phi'^{[\ell]}$

**Derivation:**

